In [1]:
import csv
import torch
print(torch.__version__)
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import numpy as np
import pandas as pd
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm
from sklearn import metrics
from sklearn import cross_validation
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
import random
import operator
%matplotlib inline

#ignore all warnings
import warnings
warnings.filterwarnings("ignore")

import datetime
from math import radians, cos, sin, asin, sqrt
from operator import itemgetter

0.2.0_4


In [3]:
class AutoEncoder(nn.Module):
    
    def __init__(self, input_layer_size, encoded_layer_size, layers, act_func, dropout_rate):
        super(AutoEncoder, self).__init__()

        if act_func == 'tanh':
            if layers == 3:
                self.encoder = nn.Sequential(
                    nn.Linear(input_layer_size, self.getLayerSize(input_layer_size, 2)),
                    nn.Tanh(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 2), encoded_layer_size),
                )
                self.decoder = nn.Sequential(
                    nn.Linear(encoded_layer_size, self.getLayerSize(input_layer_size, 2)),
                    nn.Tanh(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 2), input_layer_size),
                    nn.Sigmoid(),       # compress to a range (0, 1)
                )
            elif layers == 5:
                self.encoder = nn.Sequential(
                    nn.Linear(input_layer_size, self.getLayerSize(input_layer_size, 2)),
                    nn.Tanh(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 2), self.getLayerSize(input_layer_size, 3)),
                    nn.Tanh(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 3), encoded_layer_size),
                )
                self.decoder = nn.Sequential(
                    nn.Linear(encoded_layer_size, self.getLayerSize(input_layer_size, 3)),
                    nn.Tanh(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 3), self.getLayerSize(input_layer_size, 2)),
                    nn.Tanh(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 2), input_layer_size),
                    nn.Sigmoid(),       # compress to a range (0, 1)
                )
            elif layers == 7:
                self.encoder = nn.Sequential(
                    nn.Linear(input_layer_size, self.getLayerSize(input_layer_size, 2)),
                    nn.Tanh(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 2), self.getLayerSize(input_layer_size, 3)),
                    nn.Tanh(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 3), self.getLayerSize(input_layer_size, 4)),
                    nn.Tanh(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 4), encoded_layer_size),
                )
                self.decoder = nn.Sequential(
                    nn.Linear(encoded_layer_size, self.getLayerSize(input_layer_size, 4)),
                    nn.Tanh(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 4), self.getLayerSize(input_layer_size, 3)),
                    nn.Tanh(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 3), self.getLayerSize(input_layer_size, 2)),
                    nn.Tanh(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 2), input_layer_size),
                    nn.Sigmoid(),       # compress to a range (0, 1)
                )
            elif layers == 9:
                self.encoder = nn.Sequential(
                    nn.Linear(input_layer_size, self.getLayerSize(input_layer_size, 2)),
                    nn.Tanh(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 2), self.getLayerSize(input_layer_size, 3)),
                    nn.Tanh(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 3), self.getLayerSize(input_layer_size, 4)),
                    nn.Tanh(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 4), self.getLayerSize(input_layer_size, 5)),
                    nn.Tanh(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 5), encoded_layer_size),
                )
                self.decoder = nn.Sequential(
                    nn.Linear(encoded_layer_size, self.getLayerSize(input_layer_size, 5)),
                    nn.Tanh(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 5), self.getLayerSize(input_layer_size, 4)),
                    nn.Tanh(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 4), self.getLayerSize(input_layer_size, 3)),
                    nn.Tanh(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 3), self.getLayerSize(input_layer_size, 2)),
                    nn.Tanh(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 2), input_layer_size),
                    nn.Sigmoid(),       # compress to a range (0, 1)
                )
            

        if act_func == 'elu':
    
            if layers == 3:
                self.encoder = nn.Sequential(
                    nn.Linear(input_layer_size, self.getLayerSize(input_layer_size, 2)),
                    nn.ELU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 2), encoded_layer_size),
                )
                self.decoder = nn.Sequential(
                    nn.Linear(encoded_layer_size, self.getLayerSize(input_layer_size, 2)),
                    nn.ELU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 2), input_layer_size),
                    nn.Sigmoid(),       # compress to a range (0, 1)
                )
            elif layers == 5:
                self.encoder = nn.Sequential(
                    nn.Linear(input_layer_size, self.getLayerSize(input_layer_size, 2)),
                    nn.ELU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 2), self.getLayerSize(input_layer_size, 3)),
                    nn.ELU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 3), encoded_layer_size),
                )
                self.decoder = nn.Sequential(
                    nn.Linear(encoded_layer_size, self.getLayerSize(input_layer_size, 3)),
                    nn.ELU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 3), self.getLayerSize(input_layer_size, 2)),
                    nn.ELU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 2), input_layer_size),
                    nn.Sigmoid(),       # compress to a range (0, 1)
                )
            elif layers == 7:
                self.encoder = nn.Sequential(
                    nn.Linear(input_layer_size, self.getLayerSize(input_layer_size, 2)),
                    nn.ELU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 2), self.getLayerSize(input_layer_size, 3)),
                    nn.ELU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 3), self.getLayerSize(input_layer_size, 4)),
                    nn.ELU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 4), encoded_layer_size),
                )
                self.decoder = nn.Sequential(
                    nn.Linear(encoded_layer_size, self.getLayerSize(input_layer_size, 4)),
                    nn.ELU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 4), self.getLayerSize(input_layer_size, 3)),
                    nn.ELU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 3), self.getLayerSize(input_layer_size, 2)),
                    nn.ELU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 2), input_layer_size),
                    nn.Sigmoid(),       # compress to a range (0, 1)
                )
            elif layers == 9:
                self.encoder = nn.Sequential(
                    nn.Linear(input_layer_size, self.getLayerSize(input_layer_size, 2)),
                    nn.ELU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 2), self.getLayerSize(input_layer_size, 3)),
                    nn.ELU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 3), self.getLayerSize(input_layer_size, 4)),
                    nn.ELU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 4), self.getLayerSize(input_layer_size, 5)),
                    nn.ELU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 5), encoded_layer_size),
                )
                self.decoder = nn.Sequential(
                    nn.Linear(encoded_layer_size, self.getLayerSize(input_layer_size, 5)),
                    nn.ELU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 5), self.getLayerSize(input_layer_size, 4)),
                    nn.ELU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 4), self.getLayerSize(input_layer_size, 3)),
                    nn.ELU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 3), self.getLayerSize(input_layer_size, 2)),
                    nn.ELU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 2), input_layer_size),
                    nn.Sigmoid(),       # compress to a range (0, 1)
                )
        
        if act_func == 'relu':
    
            if layers == 3:
                self.encoder = nn.Sequential(
                    nn.Linear(input_layer_size, self.getLayerSize(input_layer_size, 2)),
                    nn.ReLU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 2), encoded_layer_size),
                )
                self.decoder = nn.Sequential(
                    nn.Linear(encoded_layer_size, self.getLayerSize(input_layer_size, 2)),
                    nn.ReLU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 2), input_layer_size),
                    nn.Sigmoid(),       # compress to a range (0, 1)
                )
            elif layers == 5:
                self.encoder = nn.Sequential(
                    nn.Linear(input_layer_size, self.getLayerSize(input_layer_size, 2)),
                    nn.ReLU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 2), self.getLayerSize(input_layer_size, 3)),
                    nn.ReLU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 3), encoded_layer_size),
                )
                self.decoder = nn.Sequential(
                    nn.Linear(encoded_layer_size, self.getLayerSize(input_layer_size, 3)),
                    nn.ReLU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 3), self.getLayerSize(input_layer_size, 2)),
                    nn.ReLU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 2), input_layer_size),
                    nn.Sigmoid(),       # compress to a range (0, 1)
                )
            elif layers == 7:
                self.encoder = nn.Sequential(
                    nn.Linear(input_layer_size, self.getLayerSize(input_layer_size, 2)),
                    nn.ReLU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 2), self.getLayerSize(input_layer_size, 3)),
                    nn.ReLU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 3), self.getLayerSize(input_layer_size, 4)),
                    nn.ReLU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 4), encoded_layer_size),
                )
                self.decoder = nn.Sequential(
                    nn.Linear(encoded_layer_size, self.getLayerSize(input_layer_size, 4)),
                    nn.ReLU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 4), self.getLayerSize(input_layer_size, 3)),
                    nn.ReLU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 3), self.getLayerSize(input_layer_size, 2)),
                    nn.ReLU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 2), input_layer_size),
                    nn.Sigmoid(),       # compress to a range (0, 1)
                )
            elif layers == 9:
                self.encoder = nn.Sequential(
                    nn.Linear(input_layer_size, self.getLayerSize(input_layer_size, 2)),
                    nn.ReLU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 2), self.getLayerSize(input_layer_size, 3)),
                    nn.ReLU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 3), self.getLayerSize(input_layer_size, 4)),
                    nn.ReLU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 4), self.getLayerSize(input_layer_size, 5)),
                    nn.ReLU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 5), encoded_layer_size),
                )
                self.decoder = nn.Sequential(
                    nn.Linear(encoded_layer_size, self.getLayerSize(input_layer_size, 5)),
                    nn.ReLU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 5), self.getLayerSize(input_layer_size, 4)),
                    nn.ReLU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 4), self.getLayerSize(input_layer_size, 3)),
                    nn.ReLU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 3), self.getLayerSize(input_layer_size, 2)),
                    nn.ReLU(),
                    nn.Dropout(dropout_rate),
                    nn.Linear(self.getLayerSize(input_layer_size, 2), input_layer_size),
                    nn.Sigmoid(),       # compress to a range (0, 1)
                )
        
        
        
#         self.encoder = nn.Sequential(
#             nn.Linear(input_layer_size, 200),
#             nn.ELU(),
#             nn.Linear(200, 100),
#             nn.ELU(),
#             nn.Dropout(0.25),
#             nn.Linear(100, 50),
#             nn.ELU(),
#             nn.Dropout(0.25),
#             nn.Linear(50, 25),
#             nn.ELU(),
#             nn.Linear(25, 10),
#         )
#         self.decoder = nn.Sequential(
#             nn.Linear(10, 25),
#             nn.ELU(),
#             nn.Linear(25, 50),
#             nn.ELU(),
#             nn.Dropout(0.25),
#             nn.Linear(50, 100),
#             nn.ELU(),
#             nn.Dropout(0.25),
#             nn.Linear(100, 200),
#             nn.ELU(),
#             nn.Linear(200, input_layer_size),
#             nn.Sigmoid(),       # compress to a range (0, 1)
#         )

    def getLayerSize(self, input_layer_size, layer_number):
        n = int(input_layer_size/(2**(layer_number-1)))
        if n < 10:
            n = 10
        return n
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [4]:
class AETrainer():
    
    def __init__(self, input_layer_size, encoded_layer_size, layers, act_func, dropout_rate, epoch=200, batch_size=50, lr=0.001):
        
        # Create autoencoder object
        self.autoencoder = AutoEncoder(input_layer_size, encoded_layer_size, layers, act_func, dropout_rate).cuda()
        
        # Define hyper parameters
        self.EPOCH = epoch
        self.BATCH_SIZE = batch_size
        self.LR = lr         
        
        # Define optimiser
        self.optimizer = torch.optim.Adam(self.autoencoder.parameters(), lr=self.LR)
        self.loss_func = nn.MSELoss()
        self.loss_list = []
        
        # Define input layer size
        self.input_layer_size = input_layer_size
        
        

    def set_data(self, x_data, y_data, raw_scores=False):
        x_data = torch.FloatTensor(x_data)
        if raw_scores:
            y_data = torch.FloatTensor(y_data)
        else:
            y_data = torch.IntTensor(y_data)
        # Data Loader for easy mini-batch return in training
        self.train = Data.TensorDataset(x_data, y_data)
        self.train_loader = Data.DataLoader(self.train, batch_size=self.BATCH_SIZE, shuffle=True)

    
    def train_model(self):
        for epoch in range(self.EPOCH):
            for step, (x, y) in enumerate(self.train_loader):
                
                b_x = Variable(x.view(-1, self.input_layer_size)).cuda()  
                b_y = Variable(x.view(-1, self.input_layer_size)).cuda()  
                b_label = Variable(y).cuda()               # batch label

                encoded, decoded = self.autoencoder(b_x)

                loss = self.loss_func(decoded, b_y)      # mean square error
                self.optimizer.zero_grad()               # clear gradients for this training step
                loss.backward()                     # backpropagation, compute gradients
                self.optimizer.step()                    # apply gradients

                self.loss_list.append(loss.data[0])


    def encode_data(self, x_data):
        x_data = torch.FloatTensor(x_data)
        x_var = Variable(x_data.view(-1, self.input_layer_size).type(torch.FloatTensor)).cuda()
        encoded_data, _ = self.autoencoder(x_var)
        return encoded_data.cpu()
        
    
    def plot_losses(self):
            plt.plot(self.loss_list)
            plt.ylabel('MSE')
            plt.xlabel('Epoch')
            plt.show()
